# QuickStart

In this Notebook we'll run Archai's [Quickstart](https://microsoft.github.io/archai/getting_started/quick_start.html) example on Azure Machine Learning.

## Prerequisites

- Python 3.7 or later
- An Azure subscription
- An Azure Resource Group
- An Azure Machine Learning [Workspace](https://learn.microsoft.com/en-us/azure/machine-learning/quickstart-create-resources#create-the-workspace)

### Requirements

In [ ]:
%pip install azure-ai-ml azure-identity 
%pip install jinja2
%pip install archai

In [ ]:
from pathlib import Path

from IPython.display import display, Image
from IPython.core.display import HTML

from azure.ai.ml import Output

import archai.common.azureml_helper as aml_helper
import archai.common.notebook_helper as nb_helper

### Get a handle to the workspace

We load the workspace from a workspace [configuration file](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-environment#local-and-dsvm-only-create-a-workspace-configuration-file).

In [ ]:
ml_client = aml_helper.get_aml_client_from_file()

### Create a compute cluster

We provision a Linux [compute cluster](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python) fos this Notebook. See the [full list](https://azure.microsoft.com/en-ca/pricing/details/machine-learning/) on VM sizes and prices.

In [ ]:
cpu_compute_name = "nas-cpu-cluster-D14-v2"
aml_helper.create_compute_cluster(ml_client, cpu_compute_name)

### Create an environment based on a YAML file

In [ ]:
archai_job_env = aml_helper.create_environment_from_file(ml_client, conda_file="conda.yaml")

### Create job

In [ ]:
job = aml_helper.create_job(experiment_name="archai_quickstart",
                            display_name="Archai's QuickStart",
                            compute_name=cpu_compute_name,
                            environment=f"{archai_job_env.name}:{archai_job_env.version}",
                            code="main.py",
                            cli_command="python main.py --output_dir ${{outputs.output_path}}",
                            outputs=dict(
                                output_path=Output(type="uri_folder", mode="rw_mount")
                            ))

### Run job

In [ ]:
quickstart_job = aml_helper.run_job(ml_client, job)

### Stream logs of the job

In [ ]:
aml_helper.stream_job_logs(ml_client, quickstart_job)

### Download job's output

In [ ]:
output_name = "output_path"
download_path = "output"

aml_helper.download_job_output(ml_client, job_name=quickstart_job.name, output_name=output_name, download_path=download_path)

downloaded_folder = Path(download_path) / "named-outputs" / output_name

### Show Pareto Frontiers

In [ ]:
param_vs_latency_img = Image(filename=downloaded_folder / "pareto_non_embedding_params_vs_onnx_latency.png")
display(param_vs_latency_img)

In [ ]:
param_vs_memory_img = Image(filename=downloaded_folder / "pareto_non_embedding_params_vs_onnx_memory.png")
display(param_vs_memory_img)

In [ ]:
latency_vs_memory_img = Image(filename=downloaded_folder / "pareto_onnx_latency_vs_onnx_memory.png")
display(latency_vs_memory_img)

### Show search state of the last iteration

In [ ]:
df = nb_helper.get_search_csv(downloaded_folder)
csv_as_html = nb_helper.get_csv_as_stylized_html(df)
display(HTML(csv_as_html))